# Testing Main Functionality #1: Gas speciation

## Goals
1. Take in simplified fluid composition and speciate it given P, T, fO2 conditions
2. Take in complex fluid composition and speciate it given P, T, fO2 conditions
3. Ensure that an equivalent complex and simplified fluid give the same speciation results
4. Ensure fO2 being held constant at user defined value when a fluid is speciated

In [3]:
import tavernmag as tv

In [4]:
# create a MagmaticFluid object with a simple gas composition
myfluid = tv.MagmaticFluid(composition={'H2O':79, 'CO2': 15, 'S': 6}, units='wtpercent', default_units='wtpercent')

# speciate given P, T, fO2 conditions
specs = tv.calculate_speciation(sample=myfluid, pressure=1000, temperature=1000, fO2_buffer='QFM', fO2_delta=1).result
spec_comp = specs.get_composition()
print("\n")
print("spec_comp, input units")
print(spec_comp)



spec_comp, input units
CO     4.919336e-02
CO2    1.520274e+01
H2     3.395537e-02
H2O    7.751130e+01
H2S    5.595435e+00
S2     8.003605e-02
SO2    1.527344e+00
O2     1.679701e-11
dtype: float64


In [5]:
# # in wt% space
# spec_comp = specs.get_composition(units='wtpercent')
# S_tot = (spec_comp['S2'] * tv.core.oxideMass["S"]/tv.core.oxideMass["S2"] +
#          spec_comp['SO2'] * tv.core.oxideMass["S"]/tv.core.oxideMass["SO2"] +
#          spec_comp['H2S'] * tv.core.oxideMass["S"]/tv.core.oxideMass["H2S"]) # I'm so confused. Do I include this term?

# # Am I just going crazy and I've calculated the "tot" values incorrectly in wt% space? I could try converting
# # to molfraction and doing it mol-wise (e.g., SO2 is one mol S annd one mol O2...)

# CO2_tot = (spec_comp['CO2'] +
#            spec_comp['CO'] * tv.core.oxideMass["CO2"]/tv.core.oxideMass["CO"])

# H2O_tot = (spec_comp['H2O'] +
#            spec_comp['H2'] * tv.core.oxideMass["H2O"]/tv.core.oxideMass["H2"] +
#            spec_comp['H2S'] * tv.core.oxideMass["H2O"]/tv.core.oxideMass["H2S"])

# print("S_tot input: " + str(myfluid.get_composition(species="S")))
# print("S_tot after speciation: " + str(S_tot))
# print("\n")
# print("CO2_tot input: " + str(myfluid.get_composition(species="CO2")))
# print("CO2_tot after speciation: " + str(CO2_tot))
# print("\n")
# print("H2O_tot input: " + str(myfluid.get_composition(species="H2O")))
# print("H2O_tot after speciation: " + str(H2O_tot))
# print("\n")
# print("Input total: " + str(sum(myfluid.get_composition())))
# print("Total after speciation: " + str(S_tot + CO2_tot + H2O_tot))

# in molfraction space
spec_comp = specs.get_composition(units='molfrac')
S_tot = (spec_comp['S2'] * 2 +
         spec_comp['SO2'] +
         spec_comp['H2S'])

CO2_tot = (spec_comp['CO2'] +
           spec_comp['CO'])

H2O_tot = (spec_comp['H2O'] +
           spec_comp['H2'] +
           spec_comp['H2S'])

recalcd_comp = tv.MagmaticFluid({"S": S_tot,
                                 "CO2": CO2_tot,
                                 "H2O": H2O_tot},
                                 units='molfrac',
                                 default_units='molfrac',
                                 default_normalization='standard')
S_tot = recalcd_comp.get_composition(species="S")
CO2_tot = recalcd_comp.get_composition(species="CO2")
H2O_tot = recalcd_comp.get_composition(species="H2O")

print("S_tot input: " + str(myfluid.get_composition(species="S", units='molfrac')))
print("S_tot after speciation: " + str(S_tot))
print("\n")
print("CO2_tot input: " + str(myfluid.get_composition(species="CO2", units='molfrac')))
print("CO2_tot after speciation: " + str(CO2_tot))
print("\n")
print("H2O_tot input: " + str(myfluid.get_composition(species="H2O", units='molfrac')))
print("H2O_tot after speciation: " + str(H2O_tot))
print("\n")
print("Input total: " + str(sum(myfluid.get_composition(units='molfrac'))))
print("Total after speciation: " + str(S_tot + CO2_tot + H2O_tot))

S_tot input: 0.03808524856648267
S_tot after speciation: 0.037919328117160735


CO2_tot input: 0.06937079614202833
CO2_tot after speciation: 0.06927658241035987


H2O_tot input: 0.8925439552914891
H2O_tot after speciation: 0.8928040894724794


Input total: 1.0000000000000002
Total after speciation: 1.0


In [6]:
spec_simpl = specs.get_simplified_fluid_composition()
print(spec_simpl)
print(sum(spec_simpl))

H2O    78.999994
CO2    15.000007
S       5.999999
dtype: float64
100.00000000000001


Fugacities

In [7]:
# Fugacities
fugs = tv.calculate_fugacities(sample=myfluid, pressure=1000, temperature=1000, fO2_buffer='QFM', fO2_delta=1,
                               opt='gekko').result
fugs

{'CO': 0.972348617383240,
 'CO2': 128.026283528372,
 'H2': 6.2979978992,
 'H2O': 1221.5309149875247,
 'H2S': 58.247579333992356,
 'O2': 1.286291018879933e-10,
 'S2': 0.45409787711,
 'SO2': 8.542068011858877}

K values

In [8]:
# K-vals
K_vals = tv.calculate_equilibrium_constants(temperature=1000).result
K_vals

{'CO2': 11609339.690689063,
 'H2O': 17101426.28201502,
 'H2S': 13.7246296563283,
 'SO2': 98548280692.69196}

Fugacity coefficients (gammas)

In [9]:
# fugacity coefficients (gammas)

gammas = tv.calculate_fugacity_coefficients(temperature=1000, pressure=1000).result
gammas

{'CO': 1.2557199322943073,
 'CO2': 1.1766812493927399,
 'H2': 1.1871749799085225,
 'H2O': 0.9013716131009993,
 'H2S': 1.1270119924703847,
 'O2': 1.1900211579833693,
 'S2': 1.1551947347375644,
 'SO2': 1.137584896298915}

In [10]:
specs2 = tv.calculate_speciation(sample=specs, pressure=1000, temperature=1000, fO2_buffer='QFM', fO2_delta=1).result

In [11]:
specs2.get_composition()

CO     4.919338e-02
CO2    1.520275e+01
H2     3.395537e-02
H2O    7.751129e+01
H2S    5.595434e+00
S2     8.003603e-02
SO2    1.527344e+00
O2     1.679701e-11
dtype: float64

In [12]:
fugs2 = tv.calculate_fugacities(sample=specs, pressure=1000, temperature=1000, fO2_buffer='QFM', fO2_delta=1,
                               opt='gekko').result
fugs2

{'CO': 0.972349137277660,
 'CO2': 128.026351981342,
 'H2': 6.2979978166,
 'H2O': 1221.5308989668056,
 'H2S': 58.247571152148005,
 'O2': 1.286291018879933e-10,
 'S2': 0.45409776145,
 'SO2': 8.542066924014495}

In [13]:
spec2_simpl = specs2.get_simplified_fluid_composition()
spec2_simpl

H2O    78.999988
CO2    15.000014
S       5.999998
dtype: float64

In [14]:
specs3 = tv.calculate_speciation(sample=specs2, pressure=1000, temperature=1000, fO2_buffer='QFM', fO2_delta=1).result
specs3.get_composition()

CO     4.919340e-02
CO2    1.520275e+01
H2     3.395536e-02
H2O    7.751129e+01
H2S    5.595433e+00
S2     8.003600e-02
SO2    1.527344e+00
O2     1.679701e-11
dtype: float64